## Softmax回归_Pytorch简洁实现
### 使用单层神经网络

In [11]:
import torch
import torchvision
from torch import nn
from torch.nn import init
import numpy as np
import sys

### 从Fashion-MINST读取数据

In [12]:
def load_data_fashion_mnist(batch_size, resize=None, root='~/Datasets/FashionMNIST'):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)
    if sys.platform.startswith('win'):
        num_workers = 0  # 0表示不用额外的进程来加速读取数据
    else:
        num_workers = 4
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_iter, test_iter

In [13]:
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)

### 定义模型网络

输入(batch_size,1,28,28)转换为(batch_size,784)


In [14]:
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer,self).__init__()
    def forward(self,x):
        return x.view(x.shape[0],-1)

In [15]:
num_inputs=784
num_outputs=10

net=nn.Sequential(
    FlattenLayer(),
    nn.Linear(num_inputs, num_outputs),
)

init.normal_(net[1].weight, mean=0, std=0.01)
init.constant_(net[1].bias, val=0) 

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

### softmax和交叉熵损失函数
分开定义softmax运算和交叉熵损失函数可能造成数值不稳定

In [16]:
loss = nn.CrossEntropyLoss()

### 定义优化算法

In [17]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

### 计算准确率

In [18]:
# 以下为计算测试集准确率
def test_accuracy(data_iter,net):
    acc_sum, n=0.0, 0
    for X,y in data_iter:
        acc_sum+=(net(X).argmax(dim=1)==y).float().sum().item()  #注意这里是sum不是mean 之后会÷n
        n+=y.shape[0]
    return acc_sum / n

### 训练模型

In [19]:
num_epochs=5

def train(net,train_iter,test_iter,loss,num_epochs):

    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n=0.0,0.0,0
        for X,y in train_iter:
            y_hat=net(X)
            l=loss(y_hat,y).sum()

            # w和b梯度清零
            optimizer.zero_grad()

            # 计算loss函数梯度
            l.backward()
            
            #梯度下降
            optimizer.step()
            
            # loss和精确度加和
            train_l_sum+=l.item()
            train_acc_sum+=(y_hat.argmax(dim=1)==y).sum().item()
            n+=y.shape[0]
        test_acc=test_accuracy(test_iter,net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f' 
            % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [20]:
train (net, train_iter, test_iter, loss, num_epochs)

epoch 1, loss 0.0031, train acc 0.750, test acc 0.780
epoch 2, loss 0.0022, train acc 0.813, test acc 0.811
epoch 3, loss 0.0021, train acc 0.825, test acc 0.813
epoch 4, loss 0.0020, train acc 0.832, test acc 0.823
epoch 5, loss 0.0019, train acc 0.837, test acc 0.826
